# Test de Selenium pour récupérer toutes les données d'une page Web
## Le site books.toscrape, qui correspond à un catalogue de livres réels avec leur couverture et des prix et notes fictifs.
L'obectif est de créer un fichier CSV avec toutes ces données. Pour cela, on va utiliser Selenium (pour simuler un navigateur Web), Request (pour récupérer le code HTML des différentes pages) et BeautifulSoup (pour naviguer et récupérer facilement les données que l'on souhaite à partir du code HTML).

In [85]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

import requests
from bs4 import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [/Users/fromont/.wdm/drivers/chromedriver/mac64_m1/105.0.5195.52/chromedriver] found in cache


# On utilise Selenium pour ouvrir un navigateur sur le site books.toscrape

In [86]:
url = "https://books.toscrape.com/"
driver.get(url)


# En inspectant le code HTML, on voit que sur la page d'accueil, on peut récupérer le titre, la note, le prix et l'url de l'image de la photo de couverture mais pas la catégorie du livre. On va donc utiliser Selenium pour parcourir toutes les catégories de livre (et accéder à la page suivante si nécessaire), et pour chaque catégorie, on va récupérer toutes les informations souhaitées.

In [87]:
Categories1 = driver.find_element(By.CLASS_NAME,"side_categories")
Categories1 = Categories1.text.split('\n')


In [88]:
Categories1.remove("Books")

In [89]:
Categories1

['Travel',
 'Mystery',
 'Historical Fiction',
 'Sequential Art',
 'Classics',
 'Philosophy',
 'Romance',
 'Womens Fiction',
 'Fiction',
 'Childrens',
 'Religion',
 'Nonfiction',
 'Music',
 'Default',
 'Science Fiction',
 'Sports and Games',
 'Add a comment',
 'Fantasy',
 'New Adult',
 'Young Adult',
 'Science',
 'Poetry',
 'Paranormal',
 'Art',
 'Psychology',
 'Autobiography',
 'Parenting',
 'Adult Fiction',
 'Humor',
 'Horror',
 'History',
 'Food and Drink',
 'Christian Fiction',
 'Business',
 'Biography',
 'Thriller',
 'Contemporary',
 'Spirituality',
 'Academic',
 'Self Help',
 'Historical',
 'Christian',
 'Suspense',
 'Short Stories',
 'Novels',
 'Health',
 'Politics',
 'Cultural',
 'Erotica',
 'Crime']

In [90]:
Titles= []
Prices= []
Stars = []
Img_url = []
Categories = []

def get_all(category):
  page_url = driver.current_url
  print(page_url)
  page = requests.get(page_url)
  soup = BeautifulSoup(page.text, 'html.parser')

  for i in soup.findAll('h3'):
    title=i.a.attrs['title']
    Titles.append(title)
    Categories.append(c)
  for j in soup.findAll('p', class_='price_color'):
    price=j.getText()
    Prices.append(price)
  for k in soup.findAll('p',class_='star-rating'):
    star = k.attrs['class'][1]
    Stars.append(star)
  divs = soup.findAll('div',class_ = 'image_container')
  for thumbs in divs:
    tgs=thumbs.find('img',class_='thumbnail')
    urls ='https://books.toscrape.com/'+str(tgs['src'])
    new_urls = urls.replace("../","")
    Img_url.append(new_urls)

for c in Categories1:
  driver.find_element(By.LINK_TEXT,c).click()
  get_all(c)
  T = True
  while T:
    try : 
      driver.find_element(By.LINK_TEXT,"next").click()
      get_all(c)
    except:
      T=False

https://books.toscrape.com/catalogue/category/books/travel_2/index.html
https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
https://books.toscrape.com/catalogue/category/books/mystery_3/page-2.html
https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-2.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-2.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-3.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-4.html
https://books.toscrape.com/catalogue/category/books/classics_6/index.html
https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html
https://books.toscrape.com/catalogue/category/books/romance_8/index.html
https://books.toscrape.com/catalogue/category/books/romance_8/page

In [95]:
data ={"Title":Titles,"Category":Categories,'Price':Prices,'Rating':Stars,"Img_url":Img_url} 
df = pd.DataFrame(data)
df.index+=1
df["Price"]=df["Price"].apply(lambda x : x[2:]).astype(float)
df["Rating"]=df["Rating"].replace({"Zero":0,"One":1,"Two":2,'Three':3,"Four":4,"Five":5})
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 1 to 1000
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     1000 non-null   object 
 1   Category  1000 non-null   object 
 2   Price     1000 non-null   float64
 3   Rating    1000 non-null   int64  
 4   Img_url   1000 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 39.2+ KB


In [96]:
df.head()

,Title,Category,Price,Rating,Img_url
1,It's Only the Himalayas,Travel,45.17,2,https://books.toscrape.com/media/cache/27/a5/2...
2,Full Moon over Noahâs Ark: An Odyssey to Mou...,Travel,49.43,4,https://books.toscrape.com/media/cache/57/77/5...
3,See America: A Celebration of Our National Par...,Travel,48.87,3,https://books.toscrape.com/media/cache/9a/7e/9...
4,Vagabonding: An Uncommon Guide to the Art of L...,Travel,36.94,2,https://books.toscrape.com/media/cache/d5/bf/d...
5,Under the Tuscan Sun,Travel,37.33,3,https://books.toscrape.com/media/cache/98/c2/9...


In [97]:
df.to_csv("bookstoscrap.csv")